### Import required packages

In [1]:
!pip install osmnx
!pip install rasterio[3]
!pip install rasterstats
!pip install geopandas

In [2]:
import csv
import cv2
import datetime
import ee
import ee.mapclient
import geopandas as gpd
import glob
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import osmnx as ox
import pandas as pd
import pyproj
import rasterio
import re
import shapely.geometry
import sys
import time

from matplotlib.pyplot import figure
from natsort import natsorted
from osgeo import gdal, osr, ogr
from osgeo.gdalconst import *
from rasterio.enums import Resampling
from rasterio.merge import merge
from rasterio.plot import show
from rasterstats import zonal_stats
from shapely.geometry import Polygon

%matplotlib inline
ox.config(log_console=True, use_cache=True)

### Mount Google Drive
Used to save images asynchronously from Google Earth Engine, and to optionally reference input files.

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
# Import code utilities files
import sys
sys.path.insert(0,'/content/drive/My Drive/IAAI_2022_Final_Code_Data_Public')
import gdal_images

In [10]:
# Authenticate GEE account 
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=_FFwt8cwUjbs9uf4GpH3GHa73XeOIz5gxHxvM0959NU&tc=20SuBw5HQSh0Zy49kz6vaE1Ab7bJEqBMTOgF0Gf_NIA&cc=O5iFVa8uIlZQ569KbGzQvmEOS49cf5aBZ4QkL5kXfP4

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgacJ5J8GkwA2SN5vK-tgghN1WOGF6zRR-5prIHEQHO7ILCQ0eJqeI

Successfully saved authorization token.


### Set Program Constants

In [5]:
# location of country to download images 
LOC = "Madagascar"

# .csv of coordinates to filter the downloaded images to particular regions 
FILTER_REGIONS = "/content/drive/My Drive/TEST/mnd_locations_by_hh_by_max.csv"

# output files for download coordinates 
COUNTRY_COORDS = "/content/drive/My Drive/TEST/full-country-coordinates.csv"
COUNTRY_COORDS_FILTERED = "/content/drive/My Drive/TEST/filtered-area-coordinates.csv"

# target resolution (e.g. ~0.0002) for output tiles 
RES = 0.0689655  #about 3100 images at about 520x520 resolution, only about 5 KB 
                 # 0.0689655 - about 256x256 expected for 30m resolution; changed to 0.1 b/c 12000 images; 0.1 yields about 4500 images at about 375x375

# GEE_DIR: Folder name for saving raw data from Google Earth Engine (NOTE: only top level, any //'s will just go in folder name)
  # LEAD_: for later when absolute path is needed to GEE_DIR
# INPUT_DIR: Accepts a folder name with input tifs (e.g., anything downloaded separately, not from GEE)
# INTER_DIR: A temporary directory used for processing. Should be separate from other directories
  # _REGIONAL is a separate folder for region-specific images 
# OUTPUT_DIR: Output folder directory
GEE_DIR = "TEST_GEE/"
LEAD_GEE_DIR = "/content/drive/My Drive/"
INPUT_DIR = "/content/drive/My Drive/TEST/raw_imgs/"
INPUT_DIR_REGIONAL = LEAD_GEE_DIR+GEE_DIR.split("/")[0]+" /"  # Note that a '/' at the end of GEE_DIR may lead to an ending space
BASE_NAME = "img_elevation_*"  # Choose 1 base name to get the proper # of images for 1 set (e.g., 23)

INTER_DIR = "/content/drive/My Drive/TEST/resampled_imgs_regional/"
OUTPUT_DIR = "/content/drive/My Drive/TEST/final_imgs/"

#Paths to survey, healthsites, markets, etc. point data.
HEALTH = '/content/drive/My Drive/Colab Notebooks/mnd/madagascar-health-locations/healthsites_one-hot.csv'
MARKETS = '/content/drive/My Drive/TEST/known-regional-found-markets.csv'

#Output folders.
label_dirs = ['/content/drive/My Drive/TEST/Fe/', '/content/drive/My Drive/TEST/A/', '/content/drive/My Drive/TEST/B12/']
surveys = ['/content/drive/My Drive/Colab Notebooks/mnd/mnd_features_locations_combined_aggregated_pixels_fullv2_Ferritin.csv', 
           '/content/drive/My Drive/Colab Notebooks/mnd/mnd_features_locations_combined_aggregated_pixels_fullv2_Retinol.csv',
           '/content/drive/My Drive/Colab Notebooks/mnd/mnd_features_locations_combined_aggregated_pixels_fullv2_B12.csv']
valueCols = ['Fe Deficiency (Ferritin)', 'Vitamin A Deficiency', 'B12 Deficiency']

mktBandNames = ['mkt_3-75km', 'mkt_7-5km']
mktDists = [150, 300] #3.75, 7.5 km, remember that 1 pix is 25m
healthBandNames = ['health_7-5km', 'health_17-5km']
healthDists = [300, 700] #7.5, 17.5 km

### Save Coordinates to Dowload Images From  
Saves a .csv file of a list of coordinates to reference when saving remote sensing imagery tiles for prediction. 


#### Save coordinates for entire country
The following blocks saves coordinates for an entire country, if predictions are to be run for the full area. The subsequent block downloads for specific regions as desired. 

In [ ]:
gdf = ox.geocode_to_gdf(LOC)

# included in case you need a different projection 
p_ll = pyproj.Proj(init='epsg:4326')
p_mt = pyproj.Proj(init='epsg:4326') 

mad_bounds = gdf.total_bounds
min_x, min_y, max_x, max_y = mad_bounds

sw = shapely.geometry.Point((min_x, min_y))
ne = shapely.geometry.Point((max_x, max_y))

# space between coordinates, corresponds to size of downloaded tiles 
step = RES 

# if different projection is desired 
transformed_sw = pyproj.transform(p_ll, p_mt, sw.x, sw.y) 
transformed_ne = pyproj.transform(p_ll, p_mt, ne.x, ne.y) 

gridpoints = []
x = transformed_sw[0]
while x < transformed_ne[0]:
    y = transformed_sw[1]
    while y < transformed_ne[1]:
        p = shapely.geometry.Point((x, y))
        if gdf.geometry.contains(p)[0]:
          gridpoints.append(p)
        y += step
    x += step


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/usr/local/lib/pytho

In [ ]:
# save the coordinates to a .csv file 
wtr = csv.writer(open(COUNTRY_COORDS, 'w'), delimiter=',', lineterminator='\n')

for p in gridpoints:
    wtr.writerow([p.x, p.y])

In [ ]:
# visualize the saved coordaintes (these cover the full country)
coords = pd.read_csv(COUNTRY_COORDS, header=None)
gdf = ox.geocode_to_gdf('Madagascar')
# gdf.plot()
# plt.scatter(coords[0].tolist(), coords[1].tolist(), c="Red")
# plt.figure(figsize=(20, 40))
# plt.show()

#### Save coordinates for filtered region
The following code blocks instead save coordinates for a desired subregion, for example in particular survey areas. 

In [ ]:
# FILTER_REGIONS contains coordinates to download tiles around
HH = pd.read_csv(FILTER_REGIONS,header=[0])
HHLat = HH['Latitude'].tolist()
HHLong = HH['Longitude'].tolist()

HHCoordinates = list(zip(HHLong, HHLat))
finalCoordinatesX = []
finalCoordinatesY = []

# find coordinates closest to filtered areas in original reference, keep 
for h in HHCoordinates:
    distance = []
    for p in gridpoints:
        distance.append(np.sqrt((p.x - h[0])**2 + (p.y - h[1])**2))
    minIndex = np.argmin(np.asarray(distance))
    if distance[minIndex] <= 0.0689655:
        finalCoordinatesX.append(gridpoints[minIndex].x)
        finalCoordinatesY.append(gridpoints[minIndex].y)
    else:
        print('possible error, ', h)

finalCoordinates = list(zip(finalCoordinatesX, finalCoordinatesY))
finalCoordinatesPrime = list(set(finalCoordinates))

In [ ]:
# write filtered coordinates to .csv file
wtr = csv.writer(open(COUNTRY_COORDS_FILTERED, 'w'), delimiter=',', lineterminator='\n')

for p in finalCoordinatesPrime:
    wtr.writerow([p[0], p[1]])

In [ ]:
# visualize the saved coordaintes (these cover the full country)
coords = pd.read_csv(COUNTRY_COORDS_FILTERED, header=None)
gdf = ox.geocode_to_gdf('Madagascar')

# gdf.plot()
# plt.scatter(coords[0].tolist(), coords[1].tolist(), c="Red")
# plt.figure(figsize=(20, 40))
# plt.show()

### Download Earth Engine Imagery from Selected Areas 
Using the saved coordinate areas from which to download imagery, the following code gathers the appropriate tiles from Google Earth Engine and saves to Google Drive. 

Google Earth Engine datasets are either in the form of a collection (raster stack) or single image. Each of these has different methods for downloading, which can be done using the functions download_collection and download_image respectivley. Both function calls accept the same parameters:

`download_collection(datasets, timerange, coordinates, outputFolder, scale, testRun)`

An example of a function call is included at the end of each code files.

In [ ]:
# reference the filtered regions to download from 
coords = pd.read_csv(COUNTRY_COORDS_FILTERED, header=None)

#### Define download functions 

In [ ]:
# Method to generate polygons from center points and grid sizes
def generate_polygons(coordinates, boxSize):
  polygons = []
  for c in coordinates:
    bw = boxSize[0]/2
    bh = boxSize[1]/2
    wc = c[0]
    hc = c[1]

    polygon = ee.Geometry.Polygon([
      [[wc-bw, hc-bh],
      [wc+bw, hc-bh],
      [wc+bw, hc+bh],
      [wc-bw, hc+bh],
      [wc-bw, hc-bh]
      ]
    ])

    polygons.append(polygon)
  return polygons

In [ ]:
# Special function for processing Sentinel-2 dataset
def process_sentinel(dataset, time_range, area):

    # remove clouds from images with masking
    def maskclouds(image):
        band_qa = image.select('QA60')
        cloud_m = ee.Number(2).pow(10).int()
        cirrus_m = ee.Number(2).pow(11).int()
        mask = band_qa.bitwiseAnd(cloud_m).eq(0) and(
            band_qa.bitwiseAnd(cirrus_m).eq(0))
        return image.updateMask(mask).divide(10000)

    # produce filtered image using median
    filter_image = (ee.ImageCollection(dataset).
                         filterBounds(area).
                         filterDate(time_range[0], time_range[1]).
                         filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).
                         map(maskclouds))

    sentinel_median = filter_image.median()
    # image_band = sentinel_median.select(['B4','B3','B2'])
    return sentinel_median

In [ ]:
# Download image collection (stacked rasters)
# for reference: https://developers.google.com/earth-engine/apidocs/export-image-todrive#python
# open this to check status: https://code.earthengine.google.com

def download_collection(datasets, names, timerange, coordinates, outputFolder, scale, bands):
      
  print("Exporting Image Collection to " + outputFolder + " Folder")

  # iterate through list of datasets 
  for d_i,d in enumerate(datasets):
    
    print("\nProcessing " + d + "...")
    
    # iterate through (possibly filtered) coordiantes to download from
    for c_i, c in enumerate(coordinates):

      # None means the default bands are used, varies by dataset 
      if bands[d_i] is None:
        collection = (ee.ImageCollection(d)
                  .filterDate(timerange[d_i][0], timerange[d_i][1])
                  .filterBounds(c)
                  )
        if 'fishing' in names[d_i]:
          fishingIm = (ee.ImageCollection(d)
                  .filterDate(timerange[d_i][0], timerange[d_i][1])
                  .filterBounds(c)
                  .sum()
                  .reduce(ee.Reducer.sum())
                  )
        else:
          fishingIm = None

        if 'fire' in names[d_i]:
          fireIm = (ee.ImageCollection(d)
                  .filterDate(timerange[d_i][0], timerange[d_i][1])
                  .filterBounds(c)
                  .sum()
                  )
        else:
          fireIm = None

      # Otherwise, specified bands in function call  
      else:
        collection = (ee.ImageCollection(d)
                .filterDate(timerange[d_i][0], timerange[d_i][1])
                .filterBounds(c)
                .select(bands[d_i])
                )
        if 'fishing' in names[d_i]:
          fishingIm = (ee.ImageCollection(d)
                  .filterDate(timerange[d_i][0], timerange[d_i][1])
                  .filterBounds(c)
                  .select(bands[d_i])
                  .sum()
                  .reduce(ee.Reducer.sum())
                  )
        else:
          fishingIm = None
        if 'fire' in names[d_i]:
          fireIm = (ee.ImageCollection(d)
                  .filterDate(timerange[d_i][0], timerange[d_i][1])
                  .filterBounds(c)
                  .select(bands[d_i])
                  .sum()
                  )
        else:
          fireIm = None

      # export images to drive, formatted with dataset and coordiante index in name 
      if fishingIm is not None:
        task = ee.batch.Export.image.toDrive(
              image=fishingIm.clip(c).toDouble(),
              description="img_" + str(names[d_i]) + "_0_" + str(c_i),
              folder=outputFolder,
              region=c,
              crs='EPSG:4326',
              scale=scale[d_i]
          ).start()
      elif fireIm is not None:
        task = ee.batch.Export.image.toDrive(
              image=fireIm.clip(c).toDouble(),
              description="img_" + str(names[d_i]) + "_0_" + str(c_i),
              folder=outputFolder,
              region=c,
              crs='EPSG:4326',
              scale=scale[d_i]
          ).start()
      else:
        collectionList = collection.toList(collection.size())
        collectionSize = collectionList.size().getInfo()
        if collectionSize > 1:
          print('WARNING: collection size > 1', collectionSize)

        for col in range(collectionSize):
          task = ee.batch.Export.image.toDrive(
                image=ee.Image(collectionList.get(col)).clip(c).toDouble(),
                description="img_" + str(names[d_i]) + "_" + str(col) + "_" + str(c_i),
                folder=outputFolder,
                region=c,
                crs='EPSG:4326',
                scale=scale[d_i]
            ).start()

    print("Complete")


In [ ]:
# Download single image rather than image collection 
# for reference: https://developers.google.com/earth-engine/apidocs/export-image-todrive#python
# open this to check status: https://code.earthengine.google.com 

def download_image(dataset, name, coordinates, outputFolder, scale, bands):
  
  print("Exporting Image Collection to " + outputFolder + " Folder")
  
  # iterate through and download from all coordinates 
  for ind, coordinate in enumerate(coordinates): 
    image = (ee.Image(dataset).clip(coordinate).select(bands).toDouble())

    task = ee.batch.Export.image.toDrive(
            image=image,
            description="img_" + str(name) + "_" + str(ind),
            folder=outputFolder,
            region=coordinate,
            crs='EPSG:4326',
            scale=scale,
        )
    task.start()

  print("Complete")

#### Call Download Functions 

In [ ]:
# set download parameters for collection downloads 
datasets = ['CIESIN/GPWv411/GPW_Population_Density', #population #https://developers.google.com/earth-engine/datasets/catalog/CIESIN_GPWv411_GPW_Population_Density#description
            'NASA/FLDAS/NOAH01/C/GL/M/V001', #weather https://developers.google.com/earth-engine/datasets/catalog/NASA_FLDAS_NOAH01_C_GL_M_V001
            'JAXA/ALOS/PALSAR/YEARLY/FNF', #forest-binary #https://developers.google.com/earth-engine/datasets/catalog/JAXA_ALOS_PALSAR_YEARLY_FNF
            'COPERNICUS/Landcover/100m/Proba-V-C3/Global', #landcover #https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global#bands
            'GFW/GFF/V1/fishing_hours', #fishing #https://developers.google.com/earth-engine/datasets/catalog/GFW_GFF_V1_fishing_hours #note: used sum and reducer as suggested here: https://globalfishingwatch.org/data-blog/working-with-our-public-data-google-earth-engine/ -- this means the output is for all methods of fishing, all countries/metadata, etc. - did not figure out why it was providing 41 outputs, but they are summed
            'NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG', #nighttime #https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMCFG#bands
            'FIRMS' #fire #https://developers.google.com/earth-engine/datasets/catalog/FIRMS #it has lots of nan values, so I've summed over a year so it's not all nan
            ]

names = ['population', 
         'weather', 
         'forest-binary', 
         'landcover',
         'fishing',
         'nighttime',
         'fire'
         ] 

scale = [25,
         25,
         25,
         25,
         25,
         25,
         25
         ]


bands = [None, #None means use default - only 1 here
         None, #None means use default - all 28 here
         None, #None means use default - only 1 here
         ['discrete_classification', 'forest_type'],
         None,
         ['avg_rad'], 
         ['T21'] 
         ] 

dates = [[datetime.datetime(2015, 1, 1), datetime.datetime(2016, 1, 1)], #only 2015 and 2020, so 2015
         [datetime.datetime(2017, 1, 1), datetime.datetime(2017, 1, 2)],
         [datetime.datetime(2017, 1, 1), datetime.datetime(2017, 1, 2)],
         [datetime.datetime(2017, 1, 1), datetime.datetime(2017, 1, 2)],
         [datetime.datetime(2016, 1, 1), datetime.datetime(2017, 1, 2)], #sum over full yr
         [datetime.datetime(2017, 1, 1), datetime.datetime(2017, 1, 2)],
         [datetime.datetime(2016, 1, 1), datetime.datetime(2017, 1, 2)]  #sum over full yr
         ]

centers = list(zip(coords[0].tolist(), coords[1].tolist()))
box = [0.08, 0.08]

In [ ]:
# download function calls happen here 
# NOTE: Fishing can take a long time, on the order of 2-9 hours per image
download_image(dataset='NASA/NASADEM_HGT/001', name='elevation_0', coordinates=generate_polygons(centers, [0.0689655,0.0689655]), outputFolder=GEE_DIR, scale=25, bands=['elevation']) #elevation #https://developers.google.com/earth-engine/datasets/catalog/NASA_NASADEM_HGT_001
download_image(dataset='JRC/GSW1_2/Metadata', name='water_0', coordinates=generate_polygons(centers, box), outputFolder=GEE_DIR, scale=25, bands=['detections']) #surface water #https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_2_Metadata#bands #note: skipping num observations band - could be used to exclude some points or normalize
download_image(dataset='UMD/hansen/global_forest_change_2017_v1_5', name='forest-change_0', coordinates=generate_polygons(centers, box), outputFolder=GEE_DIR, scale=25, bands=['lossyear']) #forest change #https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2017_v1_5 #note: using older version so it only goes through 2017
download_collection(datasets=datasets, names=names, timerange=dates, coordinates=generate_polygons(centers, box), outputFolder=GEE_DIR, scale=scale, bands=bands)

### Layer Rasters Into Single Predictor Stack
All remote sensing imagery is currently saved in separate rasters with possibly mulitple layers per raster. The following code stacks each band from each of these rasters into a single predictor array and saves as a raster with a uniform resolution. 

#### Define helper functions for raster stacking 

Merges raster images within folders in INPUT_DIR. Folder names will be used for output filename. 

In [30]:
def mosaicImages(directory):
    pathList = []
    rasterioObjList = []
    outName = ""
    for dirs, subdirs, files in os.walk(directory):
        outName = os.path.basename(dirs)
        for f in files:
            path = dirs + os.sep + f
            if path.endswith(".tif"):
                src = rasterio.open(path)
                pathList.append(path)
                rasterioObjList.append(src)
             
    # Create mosaic image        
    mosaic, out_t = merge(rasterioObjList)
    out_meta = rasterioObjList[0].meta.copy()
    out_meta.update({
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_t
    })
    
    # Save mosaic image
    with rasterio.open(INPUT_DIR + outName + ".tif", "w", **out_meta) as dest:
        dest.write(mosaic)
        
    # Set mosaic image band names 
    inDs = gdal.Open(pathList[0])
    mosaicDs = gdal.Open(INPUT_DIR + outName + ".tif")
    for i in range(inDs.RasterCount):
        mosaicDs.GetRasterBand(i+1).SetDescription(inDs.GetRasterBand(i+1).GetDescription())
        
    mosaicDs = None

If you wish to copy the extent and resolution of a single input file for the final stacked output, identify values using the following two functions.

In [31]:
def getExtent(path):
    raster_file = gdal.Open(path,0)
    geoTransform = raster_file.GetGeoTransform()
    minx = geoTransform[0]
    maxy = geoTransform[3]
    maxx = minx + geoTransform[1]*raster_file.RasterXSize
    miny = maxy + geoTransform[5]*raster_file.RasterYSize
    return [minx,miny,maxx,maxy]

def getResolution(path):
    reference = gdal.Open(path, 0)
    rt = reference.GetGeoTransform()
    xr = rt[1]
    yr = -rt[5]
    return [xr, yr]

Confirm that raster covers the desired output extent

In [32]:
def checkExtent(ds, TARGET_EXT):
    # Source: https://gis.stackexchange.com/questions/57834/how-to-get-raster-corner-coordinates-using-python-gdal-bindings
    
    ulx, xres, xskew, uly, yskew, yres  = ds.GetGeoTransform()
    lrx = ulx + (ds.RasterXSize * xres)
    lry = uly + (ds.RasterYSize * yres)
    
    source = osr.SpatialReference()
    source.ImportFromWkt(ds.GetProjection())

    # Target projection
    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)

    # Transformation function
    transform = osr.CoordinateTransformation(source, target)

    # Transform points
    ulx, uly, ret = transform.TransformPoint(ulx, uly)
    lrx, lry, ret = transform.TransformPoint(lrx, lry)
    
    if( ulx <= TARGET_EXT[0] and uly >= TARGET_EXT[1] and lrx >= TARGET_EXT[2] and lry <= TARGET_EXT[3]):
        return True
    else:
        return False

Edits all input rasters and mosaics to desired extent and resolution and saves products to INTER_DIR. If needed, change resampling algorithm from "gdal.GRIORA_Bilinear" to any desired value. 

In [33]:
def get_intermediate_images(INTER_DIR, INPUT_DIR, INPUT_DIR_REGIONAL, TARGET_RES, TARGET_EXT, index): 
    gdal.UseExceptions() 

    # Count total number of layers for future stacking 
    numLayers = 0

    # Check if intermediate path exists
    if not os.path.exists(INTER_DIR+str(index)+'/'):
        os.mkdir(INTER_DIR+str(index)+'/')
        
    # Iterate through input directory and find .tif files
    input_tifs = []
    for subdir, dirs, files in os.walk(INPUT_DIR):
        for d in dirs:
            mosaicImages(INPUT_DIR + d)
    for subdir, dirs, files in os.walk(INPUT_DIR):
        for f in files:
            path = subdir + os.sep + f
            if path.endswith(".tif") or path.endswith(".JP2"):
                input_tifs.append(path)
        break
    #Also do regional input, but only for current region
    for subdir, dirs, files in os.walk(INPUT_DIR_REGIONAL):
        for f in files:
            path = subdir + os.sep + f
            if path.endswith(".tif") and "_"+str(index)+".tif" in path:
                input_tifs.append(path)
        break
    
    # Resize, resample and save to intermediate dir
    for tif in input_tifs:
        inputDs = gdal.Open(tif)
        if checkExtent(inputDs, TARGET_EXT):
            numLayers += inputDs.RasterCount
            ds = gdal.Warp(str(INTER_DIR +str(index)+'/'+ os.path.basename(tif)), 
                          inputDs,
                          xRes=TARGET_RES[0], yRes=TARGET_RES[1],
                          outputBounds=TARGET_EXT,
                          resampleAlg= gdal.GRIORA_Bilinear, # implies bilinear is fine: https://gis.stackexchange.com/questions/352476/bilinear-resampling-with-gdal-leaves-holes
                          dstSRS='EPSG:4326',
                          format='GTiff'
                          )
            ds = None
            
        else:
            print(tif + " does not contain specified target extent. Excluding file from output.")
    return numLayers


Iterates through intermediate product folder. Also identifies constants for final stacked raster output.

In [34]:
# Iterate through intermediate product directory
def get_intermediate_list(INTER_DIR):
    tifs = []
    for subdir, dirs, files in os.walk(INTER_DIR):
        for f in files:
            path = subdir + os.sep + f
            if path.endswith(".tif"):
                tifs.append(path)
                
    # Get shape for output file from sample intermediate prod
    inputDs = gdal.Open(tifs[0])
    geo_transform = inputDs.GetGeoTransform()
    out_shape = inputDs.GetRasterBand(1).ReadAsArray().shape
    inputDs = None 
    return natsorted(tifs), geo_transform, out_shape

Build and export stacked raster. Exports multilayer.tif to OUTPUT_DIR. Saves band names from original files, if they exist. NOTE: if a shape != provided output shape, will crop or pad, see line 15

In [35]:
def build_raster(tifs, geo_transform, out_shape, numLayers, index):
    driver = gdal.GetDriverByName("Gtiff")
    outDataset = driver.Create(OUTPUT_DIR + "multilayer"+str(index)+".tif", out_shape[1], out_shape[0], numLayers, gdal.GDT_Float32)
    outDataset.SetGeoTransform(geo_transform)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)
    outDataset.SetProjection(srs.ExportToWkt())

    currLayer = 1
    for tif in tifs:
        tempFile = gdal.Open(tif)
        for ind in range(tempFile.RasterCount):
            fileLayerArr = tempFile.GetRasterBand(ind+1).ReadAsArray()
            if fileLayerArr.shape != out_shape:
                diff = np.asarray(fileLayerArr.shape) - np.asarray(out_shape)
                if diff[0] > 0:
                    fileLayerArr = fileLayerArr[diff[0]:, :]
                if diff[1] > 0:
                    fileLayerArr = fileLayerArr[:, diff[1]:]
                if diff[0] < 0 or diff[1] < 0:
                    fileLayerArr = np.pad(fileLayerArr, [[diff[0]*-1,0], [diff[1]*-1,0]])
            outDataset.GetRasterBand(currLayer).WriteArray(fileLayerArr)
            bandName = tempFile.GetRasterBand(ind+1).GetDescription()
            if bandName == '':
                bandName = os.path.basename(tif).split('.')[0]
            else:
                bandName = os.path.basename(tif).split('.')[0] + '_' + bandName
            outDataset.GetRasterBand(currLayer).SetDescription(bandName)
            currLayer += 1

    outDataset = None

Helper function to read in point data (e.g., csv) and create a raster.

In [14]:
def point_to_raster(imList, 
                    outPath, 
                    locsDF, 
                    out_shape=True,
                    bandName='mkt_labels_3-75km',
                    latCol='Latitude', 
                    longCol='Longitude', 
                    valueCols=None, 
                    rasterTypes=['mnd'], 
                    distance=150):
    """
    Args:
        imList: representative images (base im list, e.g.)
        outPath: where to save
        locsDF: pandas df with locations (& data if applicable)
        out_shape: set to True if want to force 1 output shape
        bandName: what the band name should be saved as (only for 
            point/valueCols)
        latCol: what is the name of the column with latitude (lat, e.g.)
        longCol: what is the name of the column with longitude
        valueCols: what is being saved (could also be Household ID, e.g.), if 
            None, assume it's just coordinates
        rasterTypes: list of the following types- 'point' (circle around the 
            provided coordinates), 'region' (just raster, values no change), 
            'mnd' (just raster, values plus 1)
        distance: circle radius (in pixels)
    """
    for i, imPath in enumerate(imList):
        index = int(os.path.basename(imPath).split('.')[0].split('_')[-1])
        dataset, imdata, bandNames, nC, nR, nB, xOrigin, yOrigin, pixelWidth, pixelHeight = gdal_images.read_gdal_image(imPath)
        if out_shape and i == 0:
            tgt_shape = imdata.shape

        if valueCols is None:
            if rasterTypes[0] == 'point':
                img = np.zeros((imdata.shape[0], imdata.shape[1]))
                for point in range(locsDF.shape[0]):
                    currentLat = locsDF.loc[locsDF.index[point], latCol]
                    currentLong = locsDF.loc[locsDF.index[point], longCol]

                    # adapted from https://gis.stackexchange.com/questions/221292/retrieve-pixel-value-with-geographic-coordinate-as-input-with-gdal/221430
                    row = int((yOrigin - currentLat) / pixelHeight)
                    col = int((currentLong - xOrigin) / pixelWidth)

                    if row+distance > 0 and col+distance > 0:
                        #img, center coordinates, radius, color, line thickness (to fill), https://www.geeksforgeeks.org/python-opencv-cv2-circle-method/
                        circleMask = np.zeros((imdata.shape[0], imdata.shape[1]))
                        circleMask = cv2.circle(circleMask, (col, row), distance, 1, -1)
                        img += circleMask
            else:
                raise ValueError

            if img.shape != tgt_shape:
                diff = np.asarray(img.shape) - np.asarray(tgt_shape)
                if diff[0] > 0:
                    img = img[diff[0]:, :]
                if diff[1] > 0:
                    img = img[:, diff[1]:]
                if diff[0] < 0 or diff[1] < 0:
                    img = np.pad(img, [[diff[0]*-1,0], [diff[1]*-1,0]])
            gdal_images.write_gdal_image(outPath+'img_'+bandName+'_0_'+str(index)+'.tif', dataset, img, [bandName], img.shape[1], img.shape[0], 1, xOrigin, yOrigin, pixelWidth, pixelHeight)

        else:
            img = np.zeros((imdata.shape[0], imdata.shape[1], len(valueCols)))
            for point in range(locsDF.shape[0]):
                for v,valueCol in enumerate(valueCols):
                    ##Confirm that these are actually in order 
                    if locsDF.loc[locsDF.index[point], 'imIndex'] == index:
                        row = int(locsDF.loc[locsDF.index[point], 'row'])
                        col = int(locsDF.loc[locsDF.index[point], 'col'])
                        img[row,col,v] = locsDF.loc[locsDF.index[point], valueCol]
                        if rasterTypes[v] == 'mnd':
                            img[row,col,v] += 1 #so we can find it!

            if img.shape != tgt_shape:
                diff = np.asarray((img.shape[0], img.shape[1])) - np.asarray(tgt_shape)
                if diff[0] > 0:
                    img = img[diff[0]:, :]
                if diff[1] > 0:
                    img = img[:, diff[1]:]
                if diff[0] < 0 or diff[1] < 0:
                    img = np.pad(img, [[diff[0]*-1,0], [diff[1]*-1,0]])
            gdal_images.write_gdal_image(outPath+'img_'+rasterTypes[0]+'_0_'+str(index)+'.tif', dataset, img, valueCols, img.shape[1], img.shape[0], len(valueCols), xOrigin, yOrigin, pixelWidth, pixelHeight)

#### Create rasters from labels, anything else

In [ ]:
baseImList = natsorted(glob.glob(INPUT_DIR_REGIONAL+BASE_NAME))
if len(baseImList) == 0:
    raise ValueError("Make sure your path is correct: ", INPUT_DIR_REGIONAL+BASE_NAME)

# Labels
for i in range(len(label_dirs)):
    point_to_raster(baseImList, label_dirs[i], pd.read_csv(surveys[i],index_col=0), valueCols=[valueCols[i], 'Region'], rasterTypes=['mnd', 'region'])

# Markets (here, only "mktSure", which contains both truth and regionally predicted)
for id, d in enumerate(mktDists):
    point_to_raster(baseImList, INPUT_DIR_REGIONAL, pd.read_csv(MARKETS), bandName=mktBandNames[id], latCol='lat', longCol='long', rasterTypes=['point'], distance=d)

# Healthcare centers (here, all healthcare centers)
for id, d in enumerate(healthDists):
    point_to_raster(baseImList, INPUT_DIR_REGIONAL, pd.read_csv(HEALTH), bandName=healthBandNames[id], latCol='Y', longCol='X', rasterTypes=['point'], distance=d)

#### Stack multiple rasters 

In [ ]:
for index, path in enumerate(baseImList):
    TARGET_RES = getResolution(path)
    TARGET_EXT = getExtent(path)

    # prints to intermediate dir
    numLayers = get_intermediate_images(INTER_DIR, INPUT_DIR, INPUT_DIR_REGIONAL, TARGET_RES, TARGET_EXT, index)
    print('numLayers',numLayers)
    tifs, geo_transform, out_shape = get_intermediate_list(INTER_DIR+str(index)+'/')
    if index == 0:
      standard_out_shape = out_shape

    # builds and exports final rasters 
    build_raster(tifs, geo_transform, standard_out_shape, numLayers, index)
    print(index, 'done')